In [35]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

from pandas_datareader import data as web 
from datetime import timedelta
from utils import getDateFormat, getDayPortfolio

pd.options.mode.chained_assignment = None 

# Get the data
df_basic = pd.read_csv('./inputs/basic.csv')
df_other = pd.read_csv('./inputs/other.csv')
df_dividend = pd.read_csv('./inputs/dividend.csv')

df_closing_all = pd.read_csv('./closingPrices.csv')
df_markets = pd.read_csv('./markets.csv')

In [36]:
# Format date fields
df_basic['date'] = pd.to_datetime(df_basic['date'])
df_other['date'] = pd.to_datetime(df_other['date'])
df_dividend['date'] = pd.to_datetime(df_dividend['date'])
df_closing_all['date'] = pd.to_datetime(df_closing_all['date'])
df_markets['date'] = pd.to_datetime(df_markets['date'])

# Get the earliest transaction date
minDate = min(df_basic["date"].min(), df_other["date"].min())

today = pd.to_datetime("today").replace(hour=0, minute=0, second=0, microsecond=0)
yesterday = (pd.to_datetime("today") - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)

# A list of all the tickers
tickers_all = df_basic['ticker'].unique()

In [37]:
##############################
#  Account for stock splits  #
##############################

allSplits = []

for ticker in tickers_all:

    # Get the split and convert to a df
    splits = yf.Ticker(ticker).splits

    df_split = []

    # If there was a split, append the df to allSplits
    if len(splits) > 0:
        df_split = pd.DataFrame({'date':splits.index, 'split':splits.values})
        df_split["ticker"] = ticker
        allSplits.append(df_split[df_split["date"] >= minDate])

allSplits = pd.concat(allSplits)

# Iterate through splits and alter share counts as necessary
for index, row in allSplits.iterrows():
    df_basic.loc[(df_basic["ticker"] == row["ticker"]) & (df_basic["date"] < row["date"]), ["shareCount"]] = (df_basic['shareCount'] * row["split"]).round(6)

# Split buys and sells into separate dataframes
df_buys = df_basic.loc[df_basic["type"] == "BUY"]
df_sells = df_basic.loc[df_basic["type"] == "SELL"]

# A list of all tickers with their currency
tickers_currency = df_buys[["ticker", "currency"]].drop_duplicates().reset_index(drop=True)

# A list of currently held tickers
tickers_current = getDayPortfolio(df_buys, df_sells)["ticker"].explode().unique() 

In [38]:
###########################
#  Calculate daily value  #
###########################

# Get daily portfolio value
dates = []
values = []

currentDate = minDate

# Move day-by-day from the first date to now
while currentDate != today:
    # Get the portfolio of holdings for this date
    df_portfolio = getDayPortfolio(df_buys, df_sells, currentDate)

    # Get the closing prices for this day
    df_relevantClosing = df_closing_all.loc[df_closing_all["date"] == currentDate]

    # Get all tickers for this day
    tickers = df_portfolio["ticker"].values.tolist()

    # Merge these together, and calculate the value of each stock    
    df_portfolio = pd.merge(df_portfolio, df_relevantClosing, on="ticker")
    df_portfolio["value"] = df_portfolio["shareCount"] * df_portfolio["close"]
    
    dates.append(currentDate)
    values.append(df_portfolio["value"].sum())

    currentDate = currentDate + timedelta(days=1)
        
df_daily = pd.DataFrame(list(zip(dates, values)),columns =['date', 'value'])

In [39]:
###########################
#  Get Cumulative Return  #
###########################

# Get the necessary buy and sell columns
df_cumBuys = df_buys[["date", "value"]]
df_cumSells = df_sells[["date", "value"]]

# Group by day
df_cumBuys = df_cumBuys.groupby(["date"]).sum()
df_cumSells = df_cumSells.groupby(["date"]).sum()

# Merge into a single df
df_cumAll = pd.merge(df_cumBuys, df_cumSells, on="date", how="outer")
df_cumAll.reset_index(inplace=True)
df_cumAll = df_cumAll.rename(columns={"value_x": "amount_bought", "value_y": "amount_sold"})

# Merge with daily
df_daily = pd.merge(df_daily, df_cumAll, on="date", how="left", sort=True)
df_daily[["amount_bought", "amount_sold"]] = df_daily[["amount_bought", "amount_sold"]]*-1
df_daily[["amount_bought", "amount_sold"]] = df_daily[["amount_bought", "amount_sold"]].fillna(0)

# Calculate cumulative buys and sells
df_daily["cumBuys"] = df_daily["amount_bought"].cumsum()
df_daily["cumSells"] = df_daily["amount_sold"].cumsum()
df_daily["amount_ITM"] = df_daily["cumBuys"] + df_daily["cumSells"]

# Calculate return
df_daily["amount_return_cum"] = df_daily["value"] - df_daily["amount_ITM"]
df_daily["percent_return_cum"] = (df_daily["amount_return_cum"]*100)/df_daily["amount_ITM"]

df_daily.drop(columns=["cumBuys", "cumSells"], inplace=True)

In [40]:
##################
#  Get deposits  #
##################

# Get the deposit rows
df_deposits = df_other.loc[df_other["type"] == "Deposit"][["value", "date"]]

# Merge with daily
df_daily = pd.merge(df_daily, df_deposits, on="date", how="outer", sort=True)
df_daily = df_daily.rename(columns={"value_x": "value", "value_y": "amount_deposited"})

######################
#  Get Daily Return  #
######################

df_daily["amount_return_day"] = df_daily["amount_return_cum"] - df_daily["amount_return_cum"].shift(1)
df_daily["percent_return_day"] = df_daily["percent_return_cum"] - df_daily["percent_return_cum"].shift(1)

####################################
#  Get Other Income and Outgoings  #
####################################

df_income = df_other.loc[(df_other["value"] > 0) & (df_other["type"] != "Deposit")][["value", "date"]]
df_outgoings = df_other.loc[(df_other["value"] < 0) & (df_other["type"] != "Deposit")][["value", "date"]]

df_daily = pd.merge(df_daily, df_outgoings, on="date", how="outer", sort=True)
df_daily = pd.merge(df_daily, df_income, on="date", how="outer", sort=True)

df_daily = df_daily.rename(columns={"value_x": "ind_out", "value_y": "ind_in"})

In [41]:
# Add market returns
df_daily = pd.merge(df_daily, df_markets, on="date", how="outer", sort=True)

# Format df
df_daily = df_daily.fillna(0)
df_daily = df_daily.round(2)

# Add ID column
df_daily["dayID"] = df_daily['date'].dt.strftime("%Y%m%d").astype(str).str.replace("-","")
df_daily = df_daily[ ['dayID'] + [ col for col in df_daily.columns if col != 'dayID' ] ]

# Export df
df_daily.to_csv('./outputs/daily_summary.csv', index=False)

In [52]:
# Generate ticker summary

sum_trades = []
amount_bought = []
quant_bought = []
amount_sold = []
quant_sold = []
quant_current = []
value_current = []

for t in tickers_all:

    # Sum the buys and sells (not subtract, as buys are negative)
    sum_trades.append(len(df_buys.loc[df_buys["ticker"] == t]) + len(df_sells.loc[df_sells["ticker"] == t]))

    # Sum the # bought and sold
    quant_bought.append(df_buys.loc[df_buys["ticker"] == t, "shareCount"].sum().round(3))
    quant_sold.append(df_sells.loc[df_sells["ticker"] == t, "shareCount"].sum().round(3))

    # Sum the £ bought and sold
    amount_bought.append(df_buys.loc[df_buys["ticker"] == t, "value"].sum()*-1)
    amount_sold.append(df_sells.loc[df_sells["ticker"] == t, "value"].sum())
    
    # Work out how many shares we're currently holding
    holding = df_buys.loc[df_buys["ticker"] == t, "shareCount"].sum().round(3) - df_sells.loc[df_sells["ticker"] == t, "shareCount"].sum().round(3)

    # Work out the value of what we're currently holding
    close = df_closing_all.loc[df_closing_all["ticker"] == t]
    quant_current.append(holding)
    value_current.append(close.loc[close['date'].idxmax()]["close"]*holding)

# Combine all these lists into a single dataframe
df_tickerSummary = pd.DataFrame({
    'ticker': tickers_all, 
    'quantity_bought': quant_bought, 
    'quantity_sold': quant_sold, 
    "amount_bought": amount_bought,
    "amount_sold": amount_sold,
    "shareCount": quant_current,
    "value": value_current
})

# asset_id	full_name sector price return dividends share_portfolio RSI_3m vol_3m COV_3m EPS market_cap beta
df_tickerSummary = df_tickerSummary.sort_values(by="ticker", ignore_index=True)
df_tickerSummary = df_tickerSummary.reset_index()
df_tickerSummary.rename(columns={"index": "assetID"}, inplace=True)
print(df_tickerSummary)

    assetID  ticker  quantity_bought  quantity_sold  amount_bought  \
0         0    AAPL         5548.709          0.000           2449   
1         1  BARC.L         1832.839          0.000           4049   
2         2    GOOG           14.366         12.356           3134   
3         3  HUKX.L           75.430          0.000           4443   
4         4     JNJ           83.299          1.441           5227   
5         5     JPM           74.476          0.940           3564   
6         6       K          107.885          0.000           4915   
7         7     MCD           80.956          0.889           6247   
8         8     PFE          353.092          0.000           7966   
9         9    PLUG         8635.643          0.000           3873   
10       10     SPY           31.976          0.517           4212   
11       11     XOM           74.683          0.000           4457   

    amount_sold  shareCount          value  
0             0    5548.709  690336.061600  

    ticker  £ Spent     # Bought  £ Sold     # Sold
0     AAPL     2449  5548.708560     0.0   0.000000
1   BARC.L     4049  1832.839003     0.0   0.000000
2     GOOG     3134    14.366176  4260.0  12.355663
3   HUKX.L     4443    75.429665     0.0   0.000000
4      JNJ     5227    83.298866   117.0   1.440549
5      JPM     3564    74.476406    27.0   0.940403
6        K     4915   107.884878     0.0   0.000000
7      MCD     6247    80.955522    63.0   0.889185
8      PFE     7966   353.091565     0.0   0.000000
9     PLUG     3873  8635.642790     0.0   0.000000
10     SPY     4212    31.975612    69.0   0.517356
11     XOM     4457    74.682719     0.0   0.000000
